# Contents
Data cleaning report for AMPD  - verifying data against eGRID

In [ ]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

In [ ]:
%matplotlib inline
import sys
import os
import time
start_time = time.time()

DATA_PATH = os.getenv('DATA_PATH')
CODE_PATH = os.getenv('CODE_PATH')
FIGURE_PATH = os.getenv('FIGURE_PATH')
sys.path.insert(0, os.path.join(CODE_PATH, "tracking_emissions"))

import pandas as pd
import numpy as np
import json
import re
import pytz
import datetime

from src.load import EGRID, AMPD

import matplotlib.pyplot as plt

import logging.config
logging.config.fileConfig(os.path.join(CODE_PATH, "tracking_emissions/src/logging.conf"))

# Report for cleaning in step 1

In [ ]:
# Load result from step 0
ampd = AMPD(step=0)

# Load egrid data
egrid_plnt = EGRID(sheet_name='PLNT16')
# Restrict EGRID to states in Con-US
print("Removing %d plants from eGRID that are in Alaska or Hawaii" % sum(egrid_plnt.df.PSTATABB.isin(['AK', 'HI'])))
egrid_plnt.df = egrid_plnt.df[~egrid_plnt.df.PSTATABB.isin(['AK', 'HI'])]

## Dropping AMPD plants that do not have enough timestamps

In [ ]:
# Drop the AMPD plants that do not have enough timestamps
x = ampd.df.loc[:, ["ORISPL_CODE", "OP_DATE_TIME"]].groupby(
    'ORISPL_CODE').count()
to_drop = x.mask(x > 8600).dropna()
print("AMPD: dropping %d plants out of %d that do not have enough timestamps" % (
        len(to_drop), len(x)))
ampd.df = ampd.df[~ampd.df.ORISPL_CODE.isin(to_drop.index.values)]

# to_drop has the number of hours that these plants have data for
# The plant we are dropping with the most hours is missing 25% of the year
display(to_drop.describe())

# Try to get more information on the plants we are dropping
drop_sum = egrid_plnt.df.loc[egrid_plnt.df.ORISPL.isin(to_drop.index),
                             ["ORISPL", "PSTATABB", "PNAME", "UTLSRVNM", "BANAME",
                              "BACODE", "PLFUELCT", "NAMEPCAP", "PLNGENAN", "PLCO2AN"]]
print("%d of these plants are referenced in eGRID" % len(drop_sum))
print("%d do not have a BACODE field" % drop_sum.BACODE.isna().sum())
print("For the 79 others, here is the split by BA:")
print(drop_sum.BACODE.value_counts())
print("\nSplit by fuel:")
print(drop_sum.PLFUELCT.value_counts())
print("\nAnnual generation divided by number of hours in a year:")
print(drop_sum.PLNGENAN.describe()/8760)
print("\nNameplate capacity")
print(drop_sum.NAMEPCAP.describe())
print("\nThe 4 plants with a nameplate capacity above 1GW:")
display(drop_sum[drop_sum.NAMEPCAP>1e3])

## AMPD plants that are not in eGRID 

In [ ]:
egrid_orispl = set(egrid_plnt.df.ORISPL.values)
ampd_orispl = set(ampd.df.ORISPL_CODE.values)
egrid_only = egrid_orispl - ampd_orispl
ampd_only = ampd_orispl - egrid_orispl
print("%d plants are in egrid but not in ampd" % len(egrid_only))
print("%d plants are in ampd but not in egrid" % len(ampd_only))

print("The ones we are dropping from AMPD:")
display(ampd.df.loc[ampd.df.ORISPL_CODE.isin(ampd_only),
                    ["ORISPL_CODE", "STATE", "CO2"]]\
        .groupby(["ORISPL_CODE", "STATE"]).sum())

# Drop the 11 AMPD plants that are not in EGRID
ampd.df = ampd.df[~ampd.df.ORISPL_CODE.isin(ampd_only)]


Plants in eGRID that are not in AMPD: by fuel type and then by fuel type and BA (sorted by number of missing GAS plants)

In [ ]:
print(egrid_plnt.df[egrid_plnt.df.ORISPL.isin(egrid_only)].groupby('PLFUELCT').ORISPL.count())

egrid_plnt.df[egrid_plnt.df.ORISPL.isin(egrid_only)]\
    .pivot_table(index='BACODE', columns='PLFUELCT', values='ORISPL',
                 aggfunc=lambda x: len(x.unique()))\
    .fillna(0).sort_values(by='GAS', ascending=False).head(20)

## Adjusting AMPD plants to match eGRID totals
See file `src/AMPD_1.py`. eGRID reports both adjusted and unadjusted data. The steps we take are:
* Reconcile AMPD with unadjusted eGRID data
* Calculate what the adjustment was on an annual basis from eGRID, and apply that on an hourly basis to AMPD

# Report for cleaning in step 2

In [ ]:
# Load AMPD data from previous step
ampd = AMPD(step=1)

# Load egrid data
egrid_ba = EGRID(sheet_name='BA16')

In [ ]:
# Add BACODE field to ampd.df and sum by BACODE and datetime
ampd.df.loc[:, "BACODE"] = ampd.df.ORISPL_CODE.map(                         
    dict(zip(egrid_plnt.df.ORISPL.values, egrid_plnt.df.BACODE.values)))                                        

# Absorb CSTO (1 plant) in BPAT
ampd.df.loc[ampd.df.BACODE == 'CSTO', 'BACODE'] = 'BPAT'
egrid_plnt.df.loc[egrid_plnt.df.BACODE == 'CSTO', 'BACODE'] = 'BPAT'
cols = [col for col in egrid_ba.df.columns if col not in ["BANAME", "BACODE"]]
egrid_ba.df.loc[egrid_ba.df.BACODE == 'BPAT', cols] += egrid_ba.df.loc[
    egrid_ba.df.BACODE == 'CSTO', cols].values
egrid_ba.df = egrid_ba.df.drop(egrid_ba.df.index[egrid_ba.df.BACODE == 'CSTO'])

cols = ["BACODE", "OP_DATE_TIME", "CO2", "SO2", "NOX"] 
ampd_ba = ampd.df.loc[:, cols].groupby(["BACODE", "OP_DATE_TIME"]).sum()       
ampd_ba.reset_index(inplace=True)

In [ ]:
def getTimezoneInfo():                                                          
    fileNm = os.path.join(DATA_PATH, "raw", "ba_tz.xlsx")
    BA_to_tz = pd.read_excel(fileNm)
    def get_offset(tz):
        if tz == "Pacific":
            return -8
        elif tz == "Central":
            return -6
        elif tz == "Arizona":
            return -7
        elif tz == "Eastern":
            return -5
        elif tz == "Mountain":
            return -7
        else:
            return 0
    BA_to_tz["offset"] = BA_to_tz.Timezone.apply(get_offset)
    BA_to_tz = dict(zip(BA_to_tz.BACODE, BA_to_tz.offset))
    return BA_to_tz
BA_to_tz = getTimezoneInfo()

In [ ]:
ampd_ba.loc[:, 'DATE_TIME_UTC'] = ampd_ba.OP_DATE_TIME
for bacode in ampd_ba.BACODE.unique():
    ampd_ba.loc[ampd_ba.BACODE==bacode, "DATE_TIME_UTC"] -= pd.DateOffset(hours=BA_to_tz[bacode])
ampd_ba.drop(columns=['OP_DATE_TIME'], inplace=True)

In [ ]:
# Compute annual totals
ampd_ba_ann = ampd_ba.groupby('BACODE').sum()

ba = egrid_ba.df.loc[:, ['BACODE', 'BACO2AN', 'BASO2AN', 'BANOXAN']]
ba.set_index('BACODE', inplace=True)
ba.sort_index(inplace=True)
ba.columns = [col.replace('BA', '').replace('AN', '') for col in ba.columns]
ba.fillna(0., inplace=True)
ba.head()

print("Extra BA rows in the EGRID BA-level data:\n")
print(ba.index.difference(ampd_ba_ann.index))
ba = ba.loc[ampd_ba_ann.index, :]
print(ba.index.difference(ampd_ba_ann.index))

timesteps = ampd_ba.loc[:, ['BACODE', 'DATE_TIME_UTC']].groupby(
    'BACODE').count()
print((~(timesteps == 8784)).sum())

In [ ]:
# try to reconcile ampd with egrid unadjusted
print(ba.index.difference(ampd_ba_ann.index))
diff = ba - ampd_ba_ann
diff.describe().style.format("{:.4g}")

# BA-level adjustment report

In [ ]:
# Keep track of the adjustments we are about to make
nPlantsPerBA = ampd.df.loc[:, ["BACODE", "ORISPL_CODE", "OP_DATE_TIME"]].groupby(
    ["BACODE", "ORISPL_CODE"]).min().reset_index().groupby("BACODE").count().ORISPL_CODE
egrid_totals = egrid_ba.df.loc[:, ['BACODE','BANAME', 'BACO2AN', 'BASO2AN', 'BANOXAN']]
egrid_totals.set_index('BACODE', inplace=True)
egrid_totals.sort_index(inplace=True)
egrid_totals.fillna(0., inplace=True)

diff2 = 100* (1 - ampd_ba_ann / ba)

adjustmentReport = pd.concat([egrid_totals, nPlantsPerBA, ampd_ba_ann, diff, diff2], axis=1, sort=True)
adjustmentReport.columns = ["NAME", "CO2_EGRID", "SO2_EGRID", "NOX_EGRID", "nPlants_AMPD",
                            "CO2_AMPD", "SO2_AMPD", "NOX_AMPD",
                            "CO2_diff", "SO2_diff", "NOX_diff",
                            "CO2_percent_diff", "SO2_percent_diff", "NOX_percent_diff"]

adjustmentReport.to_csv(os.path.join(FIGURE_PATH, "si", "adjustmentReportAMPD.csv"))

adjustmentReport.style.format(
    "{:.4g}", subset=[col for col in adjustmentReport if col != "NAME"])

In [ ]:
print("%d BAs have CO2 percent diffs higher than 5%%" % len(
    adjustmentReport[adjustmentReport.CO2_percent_diff>5]))
adjustmentReport[adjustmentReport.CO2_percent_diff>5].style.format(
    "{:.4g}", subset=[col for col in adjustmentReport if col != "NAME"])

# Investigating the differences

In [ ]:
# The difference here should be due to the plants that are not in AMPD. Check that assumption is correct:
egrid_orispl = set(egrid_plnt.df.ORISPL.values)
ampd_orispl = set(ampd.df.ORISPL_CODE.values)

# Aggregate plant to BA in EGRID
plnt_to_ba = egrid_plnt.df.groupby(['BACODE'])['PLCO2AN', 'PLSO2AN', 'PLNOXAN'].sum()
plnt_to_ba.columns = [col.replace('PL', '').replace('AN', '') for col in plnt_to_ba.columns]
plnt_to_ba.fillna(0., inplace=True)

# Aggregate plant to BA in EGRID - AMPD plants only
plnt_to_ba2 = egrid_plnt.df[egrid_plnt.df.ORISPL.isin(ampd_orispl)].groupby(
    ['BACODE'])['PLCO2AN', 'PLSO2AN', 'PLNOXAN'].sum()
plnt_to_ba2.columns = [col.replace('PL', '').replace('AN', '') for col in plnt_to_ba2.columns]
plnt_to_ba2.fillna(0., inplace=True)

# Recalculate AMPD annual sums 
ampd_ba_ann = ampd_ba.groupby('BACODE').sum()

diff1 = plnt_to_ba2 - ampd_ba_ann
print("Subset of EGRID plants that are also in AMPD, grouped by BA VS AMPD plants, grouped by BA")
print("Difference should be small - units are metric tons")
print("Diff:")
display(diff1.describe().style.format("{:.4g}"))

diff2 = plnt_to_ba - ba
print("\nEGRID PLNT-level grouped by BA VS EGRID BA level")
print("Difference should be small - units are metric tons")
print("Diff:")
display(diff2.describe())

diff3 = (plnt_to_ba2 - plnt_to_ba) / plnt_to_ba *100
print("\n% diff: EGRID PLNT-level that are also in AMPD, grouped by BA VS EGRID PLNT-level grouped by BA")
display(diff3.describe().style.format("{:.4g}"))
print("\n%d BAs have CO2 diffs bigger than 5%%:" % len(diff3[(diff3.CO2.abs()>5)]))
display(diff3[(diff3.CO2.abs()>5)].style.format("{:.4g}"))
# diff3[(diff3.CO2.abs()>5) | (diff3.SO2.abs()>5) | (diff3.NOX.abs()>5)]

plnt_to_ba3 = egrid_plnt.df[~egrid_plnt.df.ORISPL.isin(ampd_orispl)].groupby(
    ['BACODE'])['PLCO2AN', 'PLSO2AN', 'PLNOXAN'].sum()
plnt_to_ba3.columns = [col.replace('PL', '').replace('AN', '') for col in plnt_to_ba3.columns]
plnt_to_ba3.fillna(0., inplace=True)

diff4 = -plnt_to_ba3 / plnt_to_ba *100
print("Sanity check of previous result")
print("\n% diff: EGRID PLNT-level that are not in AMPD, grouped by BA VS EGRID PLNT-level grouped by BA")
display(diff4.describe().style.format("{:.4g}"))
print("\n%d BAs have CO2 diffs bigger than 5%%" % len(diff4[(diff4.CO2.abs()>5)]))
display(diff4[(diff4.CO2.abs()>5)].style.format("{:.4g}"))

## Quick notes
* 8 BAs in the second diff - all at 100% correspond to BAs that are not in AMPD at all.

# Investigate what is happening for the 11 other BAs
## What are the summary stats for the plants with emissions in eGRID that do not report to AMPD?
* Which plants/ fuels/ etc.
* How much does each type plant represent in the missing emissions?
* In particular, how much does CHP represent?

In [ ]:
ba_list = ["AVA", "BPAT", "CISO", "ERCO", "FPL", "IID", "IPCO", "ISNE", "NEVP", "NYIS", "PGE", "PSEI"]
missing_plants = egrid_plnt.df[(~egrid_plnt.df.ORISPL.isin(ampd_orispl))
              & egrid_plnt.df.BACODE.isin(ba_list)
              & ((egrid_plnt.df.PLCO2AN > 0) | (egrid_plnt.df.PLSO2AN > 0) | (egrid_plnt.df.PLNOXAN > 0))]
print("%d plants with emissions in eGRID do not report to AMPD" % len(missing_plants))
print("%d are CHP plants" % (missing_plants.CHPFLAG == 'Yes').sum())

In [ ]:
def grouper(x):
    BACODE = x.BACODE.unique()
    if len(BACODE) != 1:
        raise ValueError("Oops")
    BACODE = BACODE[0]
    
    d = {"ORISPL":x.ORISPL.count(), "CHPFLAG":(x.CHPFLAG=="Yes").sum(),
         "PLCO2AN":x.PLCO2AN.sum(), "PLSO2AN":x.PLSO2AN.sum(),
         "PLNOXAN":x.PLNOXAN.sum(),
         "PLCO2AN_r":x.PLCO2AN.sum() / ba.loc[BACODE, "CO2"],
         "PLSO2AN_r":x.PLSO2AN.sum() / ba.loc[BACODE, "SO2"],
         "PLNOXAN_r":x.PLNOXAN.sum() / ba.loc[BACODE, "NOX"]}
    return pd.Series(d)
df_tmp = missing_plants.loc[:, ["BACODE", "PLFUELCT", "ORISPL",
                       "CHPFLAG","PLCO2AN", "PLSO2AN",
                       "PLNOXAN"]].groupby([
        "BACODE", "PLFUELCT"]).apply(grouper)
print("How much do the missing plants represent?")
print("In absolute terms and as a percentage of total EGRID BA-level emissions")
formatter = {**{"Missing %s"% poll:"{:.2%}" for poll in ["CO2", "SO2", "NOX"]},
             **{"Annual %s"% poll:"{:.2g}" for poll in ["CO2", "SO2", "NOX"]}}
df_tmp.columns = ["# of plants", "# of CHP", "Annual CO2", "Annual SO2", "Annual NOX",
                  "Missing CO2", "Missing SO2", "Missing NOX"]
df_tmp.style.format(formatter)

In [ ]:
# Same as above, but drop the distinction by fuel type
df_tmp = missing_plants.loc[:, ["BACODE", "ORISPL",
                       "CHPFLAG","PLCO2AN", "PLSO2AN",
                       "PLNOXAN"]].groupby(["BACODE"]).apply(grouper)
formatter = {**{"Missing %s"% poll:"{:.2%}" for poll in ["CO2", "SO2", "NOX"]},
             **{"Annual %s"% poll:"{:.2g}" for poll in ["CO2", "SO2", "NOX"]}}
df_tmp.columns = ["# of plants", "# of CHP", "Annual CO2", "Annual SO2", "Annual NOX",
                  "Missing CO2", "Missing SO2", "Missing NOX"]
df_tmp.style.format(formatter)

## Deep dive on California plants

In [ ]:
missing_plants.head()

In [ ]:
# Get the list of missing plants in California - first 40 ordered by CO2 emissions
df_tmp = missing_plants.loc[missing_plants.BACODE=="CISO", [
        "BACODE", "ORISPL", "CHPFLAG","PLFUELCT","PLCO2AN", "PLSO2AN",
        "PLNOXAN", "PNAME"]].sort_values(by="PLCO2AN", ascending=False).head(40)
print("Printing data for %d plants, sorted by CO2" % len(df_tmp))
formatter = {"Annual %s"% poll:"{:.2g}" for poll in ["CO2", "SO2", "NOX"]}
df_tmp.columns = ["BA", "ORISPL", "CHP Flag", "Fuel", "Annual CO2", "Annual SO2",
                  "Annual NOX", "Plant name"]
df_tmp.style.format(formatter)

In [ ]:
# Get the list of missing plants in California - first 40 ordered by NOx emissions
df_tmp = missing_plants.loc[missing_plants.BACODE=="CISO", [
        "BACODE", "ORISPL", "CHPFLAG","PLFUELCT","PLCO2AN", "PLSO2AN",
        "PLNOXAN", "PNAME"]].sort_values(by="PLNOXAN", ascending=False).head(20)
formatter = {"Annual %s"% poll:"{:.2g}" for poll in ["CO2", "SO2", "NOX"]}
print("Printing data for %d plants, sorted by NOX emissions" % len(df_tmp))
print("\nSums")
df_tmp.columns = ["BA", "ORISPL", "CHP Flag", "Fuel", "Annual CO2", "Annual SO2",
                  "Annual NOX", "Plant name"]
print(df_tmp.loc[:, ["Annual CO2", "Annual SO2", "Annual NOX"]].sum())
print("\nDetail")
df_tmp.style.format(formatter)


In [ ]:
# Missing plants that are not CHP
df_tmp = missing_plants.loc[(missing_plants.BACODE=="CISO") & (missing_plants.CHPFLAG!="Yes"), [
        "BACODE", "ORISPL", "CHPFLAG","PLFUELCT","PLCO2AN", "PLSO2AN",
        "PLNOXAN", "PNAME"]].sort_values(by="PLNOXAN", ascending=False)

formatter = {"Annual %s"% poll:"{:.2g}" for poll in ["CO2", "SO2", "NOX"]}
print("Printing data for %d plants, sorted by NOX emissions" % len(df_tmp))
print("\nSums")
df_tmp.columns = ["BA", "ORISPL", "CHP Flag", "Fuel", "Annual CO2", "Annual SO2",
                  "Annual NOX", "Plant name"]
print(df_tmp.loc[:, ["Annual CO2", "Annual SO2", "Annual NOX"]].sum())
print("\nDetail")
df_tmp.style.format(formatter)

In [ ]:
# Missing plants that are in Nevada
df_tmp = missing_plants.loc[(missing_plants.BACODE=="CISO") & (missing_plants.PSTATABB=="NV"), [
        "BACODE", "ORISPL", "CHPFLAG","PLFUELCT","PLCO2AN", "PLSO2AN",
        "PLNOXAN", "PNAME"]].sort_values(by="PLNOXAN", ascending=False)

formatter = {"Annual %s"% poll:"{:.2g}" for poll in ["CO2", "SO2", "NOX"]}
print("Printing data for %d plants, sorted by NOX emissions" % len(df_tmp))
print("\nSums")
df_tmp.columns = ["BA", "ORISPL", "CHP Flag", "Fuel", "Annual CO2", "Annual SO2",
                  "Annual NOX", "Plant name"]
print(df_tmp.loc[:, ["Annual CO2", "Annual SO2", "Annual NOX"]].sum())
print("\nDetail")
df_tmp.style.format(formatter)

## Deep dive on NYIS plants

In [ ]:
# Get the list of missing plants in NYISO - ordered by CO2 emissions
df_tmp = missing_plants.loc[missing_plants.BACODE=="NYIS", [
        "BACODE", "ORISPL", "CHPFLAG","PLFUELCT","PLCO2AN", "PLSO2AN",
        "PLNOXAN", "PNAME"]].sort_values(by="PLCO2AN", ascending=False).head(20)
formatter = {"Annual %s"% poll:"{:.2g}" for poll in ["CO2", "SO2", "NOX"]}
df_tmp.columns = ["BA", "ORISPL", "CHP Flag", "Fuel", "Annual CO2", "Annual SO2",
                  "Annual NOX", "Plant name"]
df_tmp.style.format(formatter)

## Deep dive on ERCO plants

In [ ]:
# Get the list of missing plants in ERCO - ordered by CO2 emissions
df_tmp = missing_plants.loc[missing_plants.BACODE=="ERCO", [
        "BACODE", "ORISPL", "CHPFLAG","PLFUELCT","PLCO2AN", "PLSO2AN",
        "PLNOXAN", "PNAME"]].sort_values(by="PLCO2AN", ascending=False).head(20)
formatter = {"Annual %s"% poll:"{:.2g}" for poll in ["CO2", "SO2", "NOX"]}
df_tmp.columns = ["BA", "ORISPL", "CHP Flag", "Fuel", "Annual CO2", "Annual SO2",
                  "Annual NOX", "Plant name"]
df_tmp.style.format(formatter)